In [1]:
import pandas as pd
import numpy as np
import time
from geopy.geocoders import Nominatim
from tqdm import tqdm
geolocator = Nominatim(user_agent='giovani_valdrighi')

# Limpeza de dados

## Manténdo dados de São Paulo e colunas importantes

In [31]:
chunk_iter = pd.read_csv("data/MICRODADOS_ENEM_2019.csv", encoding = 'latin1', sep = ";", chunksize = 1000000)

In [32]:
total_rows = 0
chunks_filtered = []
for chunk in chunk_iter:
    total_rows += chunk.shape[0]
    chunks_filtered.append(chunk[chunk['SG_UF_RESIDENCIA'] == 'SP'])
df_sp = pd.concat(chunks_filtered)

In [33]:
print(f"Linhas totais: {total_rows}")
print(f"Colunas totais: {df_sp.shape[1]}")
print()
print("Mantendo apenas as linhas do estado de São Paulo")
print(f"Linhas restantes: {df_sp.shape[0]}")
print("Removendo colunas que não serão usadas:")
print("Colunas com informações sobre cada questão do teste")
print("TX_RESPOSTAS_CN   TX_RESPOSTAS_CH   TX_RESPOSTAS_LC   TX_RESPOSTAS_MT  \nTX_GABARITO_CN   TX_GABARITO_CH   TX_GABARITO_LC   TX_GABARITO_MT")
df_sp = df_sp.drop(columns = ['TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH','TX_RESPOSTAS_LC','TX_RESPOSTAS_MT',
                              'TX_GABARITO_CN','TX_GABARITO_CH','TX_GABARITO_LC','TX_GABARITO_MT'])
print("Colunas com códigos que não serão úteis:")
print('CO_MUNICIPIO_NASCIMENTO  CO_UF_NASCIMENTO   CO_MUNICIPIO_ESC   CO_UF_ESC\nCO_PROVA_CN   CO_PROVA_CH CO_PROVA_LC  CO_PROVA_MT')
df_sp = df_sp.drop(columns = ['CO_MUNICIPIO_NASCIMENTO', 'CO_UF_NASCIMENTO', 'CO_MUNICIPIO_ESC',
                              'CO_UF_ESC', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT'])
print(f"Colunas restantes: {df_sp.shape[1]}")

Linhas totais: 5095270
Colunas totais: 136

Mantendo apenas as linhas do estado de São Paulo
Linhas restantes: 813772
Removendo colunas que não serão usadas:
Colunas com informações sobre cada questão do teste
TX_RESPOSTAS_CN   TX_RESPOSTAS_CH   TX_RESPOSTAS_LC   TX_RESPOSTAS_MT  
TX_GABARITO_CN   TX_GABARITO_CH   TX_GABARITO_LC   TX_GABARITO_MT
Colunas com códigos que não serão úteis:
CO_MUNICIPIO_NASCIMENTO  CO_UF_NASCIMENTO   CO_MUNICIPIO_ESC   CO_UF_ESC
CO_PROVA_CN   CO_PROVA_CH CO_PROVA_LC  CO_PROVA_MT
Colunas restantes: 120


In [34]:
df_sp.to_csv("data/MICRODADOS_ENEM_SP_2019.csv", index = False)

## Obtendo latitude e longitude de municípios

In [2]:
df_sp = pd.read_csv('data/MICRODADOS_ENEM_SP_2019.csv')
df_sp.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001017053,2019,3546801,Santa Isabel,35,SP,22,M,1,3,...,A,A,A,C,A,A,D,A,B,B
1,190001019476,2019,3538204,Pinhalzinho,35,SP,19,M,1,3,...,A,A,A,B,A,A,C,A,B,A
2,190001019850,2019,3550308,São Paulo,35,SP,17,F,1,3,...,A,A,A,A,A,A,C,A,A,A
3,190001024874,2019,3548708,São Bernardo do Campo,35,SP,19,M,1,3,...,B,A,B,B,B,A,B,B,A,B
4,190001024931,2019,3549409,São Joaquim da Barra,35,SP,37,M,2,1,...,B,A,A,A,A,A,C,B,B,B


In [6]:
cities = pd.read_csv('data/municipios.csv')
cities.head()

,municipio,lat,lon
0,São Bernardo do Campo,-23.708035,-46.550675
1,São José do Rio Preto,-20.812585,-49.380421
2,São Paulo,-23.550651,-46.633382
3,Poá,-30.032500,-51.230377
4,Guará,-20.429050,-47.825007


In [7]:
cities_cols = ['NO_MUNICIPIO_ESC','NO_MUNICIPIO_NASCIMENTO', 'NO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_RESIDENCIA']
for col in cities_cols:
    cities[col] = cities['municipio']
    cities[col+'_LON'] = cities['lon']
    cities[col+'_LAT'] = cities['lat']
    df_sp = pd.merge(left = df_sp, right = cities[[col, col+'_LON', col+'_LAT']], on = col, how = 'inner').drop_duplicates()

In [25]:
df_sp.to_csv("data/MICRODADOS_ENEM_SP_2019.csv", index = False)